In [1]:
import tensorflow as tf
import numpy as np
import gym
from go_ai import data, metrics, policies
from go_ai.models import value_model
import matplotlib.pyplot as plt
import shutil
import multiprocessing as mp

# Hyperparameters

In [2]:
BOARD_SIZE = 5

In [3]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 128
NUM_EVAL_GAMES = 32

In [4]:
BATCH_SIZE = 32
LEARNING_RATE = 2e-3

In [5]:
LOAD_SAVED_MODELS = True

# Data Parameters

In [6]:
NUM_WORKERS = mp.cpu_count()

In [7]:
EPISODES_DIR = './data/'

In [8]:
WEIGHTS_DIR = 'model_weights/'
CHECKPOINT_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
TMP_WEIGHTS_PATH = WEIGHTS_DIR + 'tmp.h5'

In [9]:
DEMO_TRAJECTORY_PATH = 'logs/a_trajectory.png'

# Go Environment
Train on a small board for fast training and efficient debugging

In [10]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Preview Model

In [11]:
val_net = value_model.make_val_net(BOARD_SIZE)

In [12]:
val_net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 5, 5, 128)         7040      
_________________________________________________________________
batch_normalization (BatchNo (None, 5, 5, 128)         512       
_________________________________________________________________
re_lu (ReLU)                 (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 128)         147584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 128)         512       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 64)          7

# Initialize Weights

In [13]:
if LOAD_SAVED_MODELS:
    val_net.load_weights(CHECKPOINT_PATH)
    print("Starting from checkpoint")
else:
    val_net.save_weights(CHECKPOINT_PATH)
    print("Initialized checkpoint and temp")
    
# Sync temp with checkpoint
val_net.save_weights(TMP_WEIGHTS_PATH)

Starting from checkpoint


# Policies

In [14]:
temp_policy_args = {
    'name': 'temp',
    'mode': 'values',
    'board_size': BOARD_SIZE,
    'model_path': TMP_WEIGHTS_PATH,
}

In [15]:
checkpoint_policy_args = {
    'name': 'checkpoint',
    'mode': 'values',
    'board_size': BOARD_SIZE,
    'model_path': CHECKPOINT_PATH,
}

In [16]:
random_policy_args = {
    'name': 'random',
    'mode': 'random',
    'board_size': BOARD_SIZE,
}

In [17]:
greedy_policy_args = {
    'name': 'greedy',
    'mode': 'greedy',
    'board_size': BOARD_SIZE,
}

# Demo and Time Games

Symmetries

In [18]:
%%time
go_env.reset()
action = (1, 2)
next_state, _, _, _ = go_env.step(action)
metrics.plot_symmetries(next_state, 'logs/symmetries.jpg')

CPU times: user 120 ms, sys: 6.51 ms, total: 127 ms
Wall time: 133 ms


With replay memory

In [19]:
%%time
data.make_episodes(temp_policy_args, temp_policy_args, 1, num_workers=1, 
                   outdir=EPISODES_DIR)

Episode worker: 1it [00:01,  1.27s/it]
temp vs. temp: 100%|██████████| 1/1 [00:00<00:00, 992.26it/s, 100.0% WIN]

CPU times: user 1.66 s, sys: 105 ms, total: 1.77 s
Wall time: 1.54 s


1.0

# Train

In [ ]:
for iteration in range(ITERATIONS):
    # Optimization
    
    # Make and write out the episode data
    data.make_episodes(temp_policy_args, temp_policy_args, EPISODES_PER_ITERATION, 
                       num_workers=NUM_WORKERS, outdir=EPISODES_DIR)
    # Read in the episode data
    np_data = data.episodes_from_dir(EPISODES_DIR)
    batched_np_data = [np.array_split(datum, len(np_data[0]) // BATCH_SIZE) for datum in np_data]
    batched_mem = list(zip(*batched_np_data))

    # Optimize
    value_model.optimize_val_net(temp_policy_args, batched_mem, LEARNING_RATE)
    
    # Evaluate against checkpoint model and other baselines
    opp_win_rate = data.make_episodes(temp_policy_args, checkpoint_policy_args, 
                                      NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

    stats = f"{100*opp_win_rate:.1f}%O"

    # If it's better than the checkpoint, update
    if opp_win_rate > 0.6:
        shutil.copy(TMP_WEIGHTS_PATH, CHECKPOINT_PATH)
        print(f"{stats} Accepted new model")
        rand_win_rate = data.make_episodes(temp_policy_args, random_policy_args, 
                                       NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
        greed_win_rate = data.make_episodes(temp_policy_args, greedy_policy_args, 
                                        NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
        print(f"{100*greed_win_rate:.1f}%G {100*rand_win_rate:.1f}%R")

        # Plot samples of states and response heatmaps
        fig = metrics.gen_traj_fig(go_env, temp_policy_args)
        fig.savefig(DEMO_TRAJECTORY_PATH)
        plt.close()

    elif opp_win_rate >= 0.5:
        print(f"{stats} Continuing to train current weights")

    else:
        shutil.copy(CHECKPOINT_PATH, TMP_WEIGHTS_PATH)
        print(f"{stats} Rejected new model")

temp vs. checkpoint: 100%|██████████| 32/32 [00:13<00:00,  2.40it/s, 21.9% WIN]


21.9%O Rejected new model


temp vs. temp:   0%|          | 0/128 [00:00<?, ?it/s]

# Evaluate

Play against our AI

In [ ]:
human_policy_args = {
    'mode': 'human',
    'board_size': BOARD_SIZE,
}

In [ ]:
data.make_episodes(checkpoint_policy_args, human_policy_args, 1, num_workers=1)